*no terminal (prompt do comando) do VS Code*:  
py -3.10 -m venv .venv-deteccao  
.venv-deteccao\Scripts\activate
> pip install ipykernel jupyter  
> pip install -U ultralytics  
> pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio  
> pip install pillow pyyaml numpy

Para fazer os labels: 

> pip install -U label-studio  
> label-studio

ETAPA 1 — Separar dataset em train e val (com prints e visualização)  
Esta célula procura imagens/labels "soltos" em data/yolo/images e data/yolo/labels  
(fora das pastas train/val) e faz o split 80/20 por padrão.  
Se já houver train/val preenchidos, a célula apenas reporta e não redivide.  

In [ ]:
import os, random, shutil
from pathlib import Path
from glob import glob
from typing import List, Dict, Tuple
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# ------------------ CONFIG ------------------
BASE_DIR = Path("../data/yolo")  # ajuste para o seu caminho (ex.: r"C:\Users\Everaldo\...\data\yolo")
VAL_FRACTION = 0.2
SEED = 42
MOVE_FILES = False     # True = move; False = copia
CREATE_EMPTY_LABELS = True   # cria .txt vazio quando não existir
AUTO_RENAME_ON_DUPLICATES = False  # se True, renomeia arquivos com mesmo 'stem' antes de dividir
# --------------------------------------------

random.seed(SEED)

IMAGES_DIR = BASE_DIR / "images"
LABELS_DIR = BASE_DIR / "labels"
TRAIN_IM_DIR = IMAGES_DIR / "train"
VAL_IM_DIR   = IMAGES_DIR / "val"
TRAIN_LB_DIR = LABELS_DIR / "train"
VAL_LB_DIR   = LABELS_DIR / "val"

for d in [IMAGES_DIR, LABELS_DIR, TRAIN_IM_DIR, VAL_IM_DIR, TRAIN_LB_DIR, VAL_LB_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def is_split_path(p: Path) -> bool:
    parts = {x.lower() for x in p.parts}
    return ("train" in parts) or ("val" in parts)

In [ ]:
# 1) Coletar imagens/labels "soltos" (não dentro de train/val)
img_exts = (".jpg", ".jpeg", ".png")
all_imgs = [Path(p) for p in glob(str(IMAGES_DIR / "**" / "*"), recursive=True) if Path(p).suffix.lower() in img_exts]
all_lbls = [Path(p) for p in glob(str(LABELS_DIR / "**" / "*.txt"), recursive=True)]

unsplit_imgs = [p for p in all_imgs if not is_split_path(p)]
unsplit_lbls = [p for p in all_lbls if not is_split_path(p)]

print("=== Caminhos base ===")
print("BASE_DIR  :", BASE_DIR.resolve())
print("IMAGES_DIR:", IMAGES_DIR.resolve())
print("LABELS_DIR:", LABELS_DIR.resolve())

print("\n=== Arquivos encontrados (fora de train/val) ===")
print("Imagens  :", len(unsplit_imgs))
print("Labels   :", len(unsplit_lbls))

In [ ]:
# 2) Mapear por stem e checar duplicatas
def map_by_stem(paths: List[Path]) -> Dict[str, List[Path]]:
    d: Dict[str, List[Path]] = {}
    for p in paths:
        d.setdefault(p.stem, []).append(p)
    return d

imgs_by_stem = map_by_stem(unsplit_imgs)
lbls_by_stem = map_by_stem(unsplit_lbls)

dup_imgs = {k: v for k, v in imgs_by_stem.items() if len(v) > 1}
dup_lbls = {k: v for k, v in lbls_by_stem.items() if len(v) > 1}

if dup_imgs or dup_lbls:
    print("\n[ALERTA] Foram detectados nomes duplicados (stems repetidos).")
    print("Imagens duplicadas:", len(dup_imgs), "| Labels duplicados:", len(dup_lbls))
    example_key = next(iter(dup_imgs.keys() if dup_imgs else dup_lbls.keys()), None)
    if example_key:
        print("Exemplo de duplicata:", example_key, "→")
        print("  imgs:", [str(p) for p in imgs_by_stem.get(example_key, [])])
        print("  lbls:", [str(p) for p in lbls_by_stem.get(example_key, [])])
    if not AUTO_RENAME_ON_DUPLICATES:
        print("\n[ERRO] Existem duplicatas e AUTO_RENAME_ON_DUPLICATES=False. "
              "Renomeie manualmente ou ative AUTO_RENAME_ON_DUPLICATES=True e rode novamente.")
        # aborta antes de mexer
        raise SystemExit

In [ ]:
# 3) (Opcional) renomear duplicatas automaticamente
def safe_rename(path: Path, suffix: str) -> Path:
    new_name = f"{path.stem}_{suffix}{path.suffix}"
    new_path = path.with_name(new_name)
    i = 1
    while new_path.exists():
        new_path = path.with_name(f"{path.stem}_{suffix}_{i}{path.suffix}")
        i += 1
    path.rename(new_path)
    return new_path

if AUTO_RENAME_ON_DUPLICATES:
    print("\n[INFO] Renomeando duplicatas...")
    for k, paths in dup_imgs.items():
        for idx, p in enumerate(paths[1:], start=2):  # mantém o primeiro, renomeia do segundo em diante
            new_img = safe_rename(p, f"d{idx}")
            # renomeia o label correspondente se existir
            lbl_p = (p.parent.parent.parent / "labels" / p.relative_to(IMAGES_DIR).with_suffix(".txt")).resolve()
            # O caminho acima tenta encontrar um label com mesma estrutura relativa.
            # Se não existir nessa estrutura, tenta por nome em unsplit_lbls.
            if lbl_p.exists():
                _ = safe_rename(lbl_p, f"d{idx}")
    # recomputa mapas após renomear
    all_imgs = [Path(p) for p in glob(str(IMAGES_DIR / "**" / "*"), recursive=True) if Path(p).suffix.lower() in img_exts]
    all_lbls = [Path(p) for p in glob(str(LABELS_DIR / "**" / "*.txt"), recursive=True)]
    unsplit_imgs = [p for p in all_imgs if not is_split_path(p)]
    unsplit_lbls = [p for p in all_lbls if not is_split_path(p)]
    imgs_by_stem = map_by_stem(unsplit_imgs)
    lbls_by_stem = map_by_stem(unsplit_lbls)
    print("[OK] Duplicatas tratadas.")


In [ ]:
# 4) Criar pares (imagem -> label) e criar .txt vazio quando não houver
pairs: List[Tuple[Path, Path]] = []
missing_labels = 0
for stem, img_list in imgs_by_stem.items():
    img_path = img_list[0]            # após checagem de duplicatas, presume 1 por stem
    lbl_list = lbls_by_stem.get(stem, [])
    if lbl_list:
        lbl_path = lbl_list[0]
    else:
        lbl_path = LABELS_DIR / f"{stem}.txt"
        if CREATE_EMPTY_LABELS and not lbl_path.exists():
            lbl_path.touch()
            missing_labels += 1
    pairs.append((img_path, lbl_path))

print("\n=== Pareamento imagem→label ===")
print("Total de pares:", len(pairs))
print("Labels vazios criados:", missing_labels)
# imprime alguns exemplos
for p in pairs[:5]:
    print(" -", p[0].name, "<->", p[1].name)

In [ ]:
# 5) Se já existir estrutura train/val preenchida, não refaz split
already_has_split = any(TRAIN_IM_DIR.glob("*")) or any(VAL_IM_DIR.glob("*"))
if already_has_split:
    print("\n[INFO] Pastas train/val já possuem arquivos. Não será feito novo split.")
else:
    # 5a) Embaralhar e dividir
    stems = list(imgs_by_stem.keys())
    random.shuffle(stems)
    n_total = len(stems)
    n_val = max(1, int(VAL_FRACTION * n_total)) if n_total > 0 else 0
    val_set = set(stems[:n_val])

    # 5b) Mover/Copiar
    def transfer(src: Path, dst: Path, move=True):
        dst.parent.mkdir(parents=True, exist_ok=True)
        if move:
            shutil.move(str(src), str(dst))
        else:
            shutil.copy2(str(src), str(dst))

    moved_train = moved_val = 0
    for stem in stems:
        img_path = imgs_by_stem[stem][0]
        lbl_path = (lbls_by_stem.get(stem, [LABELS_DIR / f"{stem}.txt"]))[0]
        dst_im = (VAL_IM_DIR if stem in val_set else TRAIN_IM_DIR) / img_path.name
        dst_lb = (VAL_LB_DIR if stem in val_set else TRAIN_LB_DIR) / f"{stem}.txt"

        transfer(img_path, dst_im, move=MOVE_FILES)
        transfer(lbl_path, dst_lb, move=MOVE_FILES)
        if stem in val_set:
            moved_val += 1
        else:
            moved_train += 1

    print("\n=== Split concluído ===")
    print(f"Total: {n_total} | train: {moved_train} | val: {moved_val}")
    print(f"Método: {'move' if MOVE_FILES else 'copy'} | VAL_FRACTION={VAL_FRACTION} | seed={SEED}")

In [ ]:
# 6) Relatório final dos diretórios
def count_files(dirp: Path, exts=("*",)):
    return sum(1 for _ in dirp.glob("|".join(exts)))

train_imgs = [p for p in TRAIN_IM_DIR.glob("*") if p.suffix.lower() in img_exts]
val_imgs   = [p for p in VAL_IM_DIR.glob("*") if p.suffix.lower() in img_exts]
train_lbls = list(TRAIN_LB_DIR.glob("*.txt"))
val_lbls   = list(VAL_LB_DIR.glob("*.txt"))

print("\n=== Pós-split: contagens ===")
print("images/train:", len(train_imgs), " | labels/train:", len(train_lbls))
print("images/val  :", len(val_imgs)  , " | labels/val  :", len(val_lbls))

In [ ]:
# 7) Visualização: mostra até 2 imagens do train e 2 do val com caixas
def load_yolo_labels(label_file: Path):
    if not label_file.exists() or label_file.stat().st_size == 0:
        return []
    out = []
    with open(label_file, "r", encoding="utf-8") as f:
        for ln in f:
            parts = ln.strip().split()
            if len(parts) != 5: 
                continue
            cls, xc, yc, w, h = parts
            out.append((int(cls), float(xc), float(yc), float(w), float(h)))
    return out

def yolo_to_xyxy(box, W, H):
    cls, xc, yc, w, h = box
    x_c, y_c = xc*W, yc*H
    bw, bh = w*W, h*H
    x1, y1 = x_c - bw/2, y_c - bh/2
    x2, y2 = x_c + bw/2, y_c + bh/2
    return int(cls), max(0,int(x1)), max(0,int(y1)), min(W-1,int(x2)), min(H-1,int(y2))

def preview_with_boxes(img_path: Path, split: str):
    lbl_path = (TRAIN_LB_DIR if split=="train" else VAL_LB_DIR) / f"{img_path.stem}.txt"
    img = Image.open(img_path).convert("RGB")
    W, H = img.size
    boxes = load_yolo_labels(lbl_path)
    # plota uma figura por imagem
    plt.figure()
    plt.imshow(img)
    ax = plt.gca()
    for b in boxes:
        _, x1, y1, x2, y2 = yolo_to_xyxy(b, W, H)
        rect = Rectangle((x1, y1), x2-x1, y2-y1, fill=False, linewidth=2)
        ax.add_patch(rect)
    plt.title(f"{split}: {img_path.name} | {len(boxes)} caixas")
    plt.axis("off")

if len(train_imgs) == 0 and len(val_imgs) == 0:
    print("\n[INFO] Sem imagens para visualizar (verifique se os arquivos estão em", BASE_DIR, ").")
else:
    print("\n=== Pré-visualização (até 4 imagens) ===")
    for p in train_imgs[:2]:
        preview_with_boxes(p, "train")
    for p in val_imgs[:2]:
        preview_with_boxes(p, "val")
    print("[OK] Visualização pronta (rolar para ver as figuras).")

In [ ]:
# ETAPA 5A — Config + checagens

import os, sys, platform, json, random
from pathlib import Path
from glob import glob

import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Tenta importar PyTorch e Ultralytics
import torch
try:
    from ultralytics import YOLO
except Exception as e:
    YOLO = None
    print("Ultralytics não importado. Rode:  %pip install -U ultralytics")

# ----- Caminhos -----
ROOT = Path.cwd()
DATASET_DIR = Path("../data/yolo")
DATA_YAML = DATASET_DIR / "data.yaml"
IMAGES_DIR = DATASET_DIR / "images"
LABELS_DIR = DATASET_DIR / "labels"

print("Python:", sys.version.split()[0], "| SO:", platform.platform())
print("Torch :", torch.__version__, "| CUDA disponível?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
device = 0 if torch.cuda.is_available() else "cpu"
print("Device escolhido para treino:", device)

# ----- Verifica dataset -----
def count_files(p: Path, exts):
    return sum(1 for f in p.glob("*") if f.suffix.lower() in exts)

img_exts = (".jpg",".jpeg",".png")
train_imgs = [p for p in (IMAGES_DIR/"train").glob("*") if p.suffix.lower() in img_exts]
val_imgs   = [p for p in (IMAGES_DIR/"val").glob("*")   if p.suffix.lower() in img_exts]
train_lbls = list((LABELS_DIR/"train").glob("*.txt"))
val_lbls   = list((LABELS_DIR/"val").glob("*.txt"))

print("\n=== Dataset ===")
print("data.yaml existe? ", DATA_YAML.exists(), "→", DATA_YAML)
print("images/train:", len(train_imgs), "| labels/train:", len(train_lbls))
print("images/val  :", len(val_imgs)  , "| labels/val  :", len(val_lbls))

assert DATA_YAML.exists(), "data.yaml não encontrado."
assert len(train_imgs) > 0 and len(train_lbls) > 0, "Sem dados em train."
assert len(val_imgs)   > 0 and len(val_lbls)   > 0, "Sem dados em val."

# Mostra conteúdo do data.yaml
import yaml
with open(DATA_YAML, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)
print("\nConteúdo de data.yaml:")
print(json.dumps(cfg, indent=2, ensure_ascii=False))

# ----- Visualização de amostras com caixas -----
def load_yolo_labels(txt_path: Path):
    if not txt_path.exists() or txt_path.stat().st_size == 0:
        return []
    out = []
    with open(txt_path, "r", encoding="utf-8") as f:
        for ln in f:
            parts = ln.strip().split()
            if len(parts) == 5:
                cls, xc, yc, w, h = parts
                out.append((int(cls), float(xc), float(yc), float(w), float(h)))
    return out

def yolo_to_xyxy(box, W, H):
    cls, xc, yc, w, h = box
    x_c, y_c = xc*W, yc*H
    bw, bh = w*W, h*H
    x1, y1 = x_c - bw/2, y_c - bh/2
    x2, y2 = x_c + bw/2, y_c + bh/2
    return int(cls), max(0,int(x1)), max(0,int(y1)), min(W-1,int(x2)), min(H-1,int(y2))

print("\n=== Amostras com caixas (até 3 do train) ===")
for img_path in train_imgs[:3]:
    img = Image.open(img_path).convert("RGB")
    W, H = img.size
    lbl = (LABELS_DIR/"train"/f"{img_path.stem}.txt")
    boxes = load_yolo_labels(lbl)

    draw = ImageDraw.Draw(img)
    for b in boxes:
        _, x1, y1, x2, y2 = yolo_to_xyxy(b, W, H)
        draw.rectangle([x1, y1, x2, y2], width=2)  # sem cor explícita (usa padrão)
        draw.text((x1, max(0, y1-10)), "muda")

    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"{img_path.name} | {len(boxes)} caixas")

print("\n[OK] Checagens concluídas.")


In [ ]:
# ETAPA 5B — Treino YOLO 11 (detecção)

from ultralytics import YOLO
import torch, time, os
from pathlib import Path

# from ultralytics import settings
# settings.update({
#     "runs_dir": str(Path("../data/yolo"))
# })

# Recomendo começar com o yolo11s; depois, se precisar de mais precisão, testar yolo11m
MODEL_NAME = "yolo11s.pt"

print("Carregando modelo base:", MODEL_NAME)
model = YOLO(MODEL_NAME)

# Hiperparâmetros recomendados para mudas top-down
train_args = dict(
    data=str(DATA_YAML),
    imgsz=1280,           # ajuda para objetos pequenos
    epochs=100,
    batch=-1,             # auto-batch
    device=0 if torch.cuda.is_available() else "cpu",
    workers=2,            # deixe 0 se tiver erro no Windows com multiprocess
    patience=20,          # early stopping
    cache=True,           # carrega imagens em RAM para acelerar
    # Aumentações — leves e seguras para folhas
    mosaic=0.05,          # reduz mosaico (0.0 para desativar)
    mixup=0.0,            # desliga mixup
    perspective=0.0,      # top-down: não distorcer perspectiva
    flipud=0.0,           # não virar de cabeça para baixo
    fliplr=0.5,           # ok inverter esquerda-direita
    # Logs/plots
    plots=True,
    verbose=True
)

print("\n=== Parâmetros de treino ===")
for k,v in train_args.items():
    print(f"{k:>12}: {v}")

print("\nIniciando treino...")
t0 = time.time()
results = model.train(**train_args)
t1 = time.time()
print(f"Treino finalizado em {t1 - t0:.1f}s (aprox).")

# Onde salvou?
save_dir = None
try:
    save_dir = model.trainer.save_dir
except Exception:
    save_dir = getattr(results, "save_dir", None)

print("save_dir:", save_dir)
if save_dir:
    save_dir = Path(save_dir)
    weights_dir = save_dir / "weights"
    print("Pesos encontrados:", list(weights_dir.glob("*.pt")))
